<a href="https://colab.research.google.com/github/fernandofsilva/LSTM_Option_Pricing/blob/main/notebooks/03_modelo_dnn_subclassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#@title Carregando as bibliotecas base
import pandas as pd
import numpy as np
import tensorflow as tf

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

seaborn.set_style('whitegrid')

In [ ]:
#@title Carregando os dados
data = pd.read_csv(f'/content/drive/My Drive/Mestrado/data/dados_treino_teste.csv.gz', compression='gzip', index_col=0)
data.head()

,codigo,mercado,preco_opcao,preco_exercicio,data_vencimento,T,preco_ativo,volatilidade,taxa_juros,black_scholes,delta_black_scholes,base
2015-01-05,PETRM17,OPÇÕES DE VENDA,8.18,16.91,2015-01-19,0.039683,8.61,0.771953,0.1157,8.22,100,treino
2015-01-05,PETRM28,OPÇÕES DE VENDA,0.36,8.41,2015-01-19,0.039683,8.61,0.771953,0.1157,0.41,40,treino
2015-01-05,PETRM2,OPÇÕES DE VENDA,0.02,4.91,2015-01-19,0.039683,8.61,0.771953,0.1157,0.00,0,teste
2015-01-05,PETRM23,OPÇÕES DE VENDA,0.79,9.21,2015-01-19,0.039683,8.61,0.771953,0.1157,0.87,63,treino
2015-01-05,PETRM25,OPÇÕES DE VENDA,1.13,9.61,2015-01-19,0.039683,8.61,0.771953,0.1157,1.16,73,treino


# Engenharia das variáveis (Feature Engineering)

Essa sessão é composta da transformação dos dados para entrada na rede na rede neural. Portando, as variáveis são transformadas do seu valor original, seja para adequação dentro da rede neural ou para um melhor treinamento da rede, essas transformações são:

- Variavéis númericas: preco_exercicio, preco_ativo, foram normalizadas antes da entrada na rede
- Variavéis númericas: preco_opcao (alvo), volatilidade, taxa_juros e T não sofreram alterações
- Variável categórica mercado sofreu one hot encoding

A transformação dos dados é feita no mesmo momento que o modelo é treinado, isso é feito através de uma camada dentro do modelo, essa camada tem o nome de feature layer.

In [ ]:
#@title Pipeline de entrada dos dados
def df_to_dataset(dataframe, base, shuffle=True, batch_size=22):

    # Criar cópia do dataframe
    dataframe = dataframe.copy()

    # Filtrar a base
    dataframe = dataframe[dataframe['base'] == base]

    # Variavel alvo
    labels = dataframe.pop('preco_opcao')

    # Colunas do modelo
    cols = ['mercado', 'preco_exercicio', 'preco_ativo', 'T', 'volatilidade', 'taxa_juros']

    # Criar o td.data
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe[cols]), labels))

    # Embaralhar os dados se necessário
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))

    # Criar o batch de dados
    ds = ds.batch(batch_size)

    return ds

# Divisão da base de treino e teste
train_ds = df_to_dataset(data, base='treino')
test_ds = df_to_dataset(data, shuffle=False, base='teste')

In [ ]:
#@title Mapeamento das colunas
feature_columns = []

# Colunas númericas normalizadas
for column in ['preco_exercicio', 'preco_ativo']:
    
    mean = data.loc[data['base'] == 'treino', column].mean()
    stdev = data.loc[data['base'] == 'treino', column].std()

    feature_columns.append(tf.feature_column.numeric_column(column, normalizer_fn = lambda x: (x - mean) / stdev))

# Colunas númericas sem normalização
for column in ['T', 'volatilidade', 'taxa_juros']:

    feature_columns.append(tf.feature_column.numeric_column(column))

# Colunas categóricas
option = tf.feature_column.categorical_column_with_vocabulary_list('mercado', ['OPÇÕES DE COMPRA', 'OPÇÕES DE VENDA'])
option_one_hot = tf.feature_column.indicator_column(option)
feature_columns.append(option_one_hot)

In [ ]:
#@title Camada de transformação (feature layer)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns, name='Feature')

# Modelo

A rede é composta por uma camada de entrada de 128 nós com função de ativação relu, uma camada escondida (intermediária) com 128 nós e uma camada de saida de 1 nó

In [ ]:
#@title Model Subclassing
class DNN_Model(tf.keras.Model):
    def __init__(self):
        super(DNN_Model, self).__init__()
        self.feature = feature_layer
        self.d1 = tf.keras.layers.Dense(128, activation='relu', name='Dense_1')
        self.d2 = tf.keras.layers.Dense(128, activation='relu', name='Dense_2')
        self.d3 = tf.keras.layers.Dense(1, name='output')

    def call(self, x):
        x = self.feature(x)
        x = self.d1(x)
        x = self.d2(x)
        return self.d3(x)

# Create an instance of the model
model = DNN_Model()

In [ ]:
#@title Inicialização do modelo
# Para apresentar o sumário do modelo, primeiramente, é necessário inicializar os pesos e bias de cada camada,
# isso é feito utilizando uma amostra dos dados
element = next(iter(train_ds.take(1)))[0]
model(element)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



<tf.Tensor: shape=(22, 1), dtype=float32, numpy=
array([[-0.2524989 ],
       [-0.3590853 ],
       [-0.31574062],
       [-0.31448415],
       [-0.3426411 ],
       [-0.3382241 ],
       [-0.34591782],
       [-0.34895217],
       [-0.16876364],
       [-0.2821396 ],
       [-0.31286868],
       [-0.14739323],
       [-0.23173298],
       [-0.35521543],
       [-0.22168958],
       [-0.35518914],
       [-0.24707964],
       [-0.285274  ],
       [-0.26231378],
       [-0.3459221 ],
       [-0.22210468],
       [-0.24105029]], dtype=float32)>

In [ ]:
#@title Sumário do modelo
model.summary()

Model: "dnn__model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  0         
_________________________________________________________________
Dense_1 (Dense)              multiple                  1024      
_________________________________________________________________
Dense_2 (Dense)              multiple                  16512     
_________________________________________________________________
output (Dense)               multiple                  129       
Total params: 17,665
Trainable params: 17,665
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#@title Função de perda e otimizador
loss_object = tf.keras.losses.MeanSquaredError(reduction="auto", name="mse")
optimizer = tf.keras.optimizers.Adam(0.001)

In [ ]:
#@title Métricas
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_mae = tf.keras.metrics.MeanAbsoluteError(name='train_mae')
train_mape = tf.keras.metrics.MeanAbsolutePercentageError(name='train_mape')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_mae = tf.keras.metrics.MeanAbsoluteError(name='test_mae')
test_mape = tf.keras.metrics.MeanAbsolutePercentageError(name='test_mape')

In [ ]:
#@title Aplicando gradiente
@tf.function
def train_step(data, labels):
    with tf.GradientTape() as tape:
        predictions = model(data, training=True)
        loss = loss_object(predictions, labels)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_mae(labels, predictions)
    train_mape(labels, predictions)

In [ ]:
#@title Função de teste
@tf.function
def test_step(data, labels):
  predictions = model(data, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_mae(labels, predictions)
  test_mape(labels, predictions)

In [ ]:
#@title Treinamento do modelo
EPOCHS = 10

for epoch in range(EPOCHS):
    # Re-inicia as métricas a cada época
    train_loss.reset_states()
    train_mae.reset_states()
    train_mape.reset_states()
    test_loss.reset_states()
    test_mae.reset_states()
    test_mape.reset_states()

    for data, labels in train_ds:
        train_step(data, labels)

    for test_data, test_labels in test_ds:
        test_step(test_data, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result():.4f}, '
        f'MAE: {train_mae.result():.4f}, '
        f'MAPE: {train_mape.result():.4f}, '
        f'Test_Loss: {test_loss.result():.4f}, '
        f'Test_MAE: {test_mae.result():.4f}, '
        f'Test_MAPE: {test_mape.result():.4f}'
        )

Epoch 1, Loss: 0.1431, MAE: 0.1920, MAPE: 174.0132, Test_Loss: 0.0658, Test_MAE: 0.1391, Test_MAPE: 104.8454
Epoch 2, Loss: 0.0607, MAE: 0.1451, MAPE: 92.7746, Test_Loss: 0.0651, Test_MAE: 0.1444, Test_MAPE: 67.1575
Epoch 3, Loss: 0.0576, MAE: 0.1357, MAPE: 69.4060, Test_Loss: 0.0566, Test_MAE: 0.1217, Test_MAPE: 52.8496
Epoch 4, Loss: 0.0561, MAE: 0.1311, MAPE: 57.8929, Test_Loss: 0.0604, Test_MAE: 0.1279, Test_MAPE: 45.9583
Epoch 5, Loss: 0.0549, MAE: 0.1287, MAPE: 53.9824, Test_Loss: 0.0603, Test_MAE: 0.1318, Test_MAPE: 50.7652
Epoch 6, Loss: 0.0537, MAE: 0.1262, MAPE: 53.2335, Test_Loss: 0.0544, Test_MAE: 0.1144, Test_MAPE: 42.3414
Epoch 7, Loss: 0.0531, MAE: 0.1247, MAPE: 51.4236, Test_Loss: 0.0606, Test_MAE: 0.1227, Test_MAPE: 47.5844
Epoch 8, Loss: 0.0525, MAE: 0.1236, MAPE: 50.6328, Test_Loss: 0.0551, Test_MAE: 0.1244, Test_MAPE: 58.2162
Epoch 9, Loss: 0.0517, MAE: 0.1219, MAPE: 48.9811, Test_Loss: 0.0540, Test_MAE: 0.1198, Test_MAPE: 50.4070
Epoch 10, Loss: 0.0514, MAE: 0.1211